In [1]:
import os
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go 
from datetime import datetime
from email.encoders import encode_base64

dt_string = datetime.now().strftime('%y%m%d')
f3_name ='220119-0827-f3-output'
f3_producto = pd.read_csv(f'input_planillas/{f3_name}.csv', sep=';')

In [2]:
f3_producto["fecha_reserva"] = f3_producto["fecha_reserva"].replace(["ene", "abr", "ago", "dic"], ["jan", "apr", "aug", "dec"], regex=True)  # cambio idioma mes
f3_producto["fecha_reserva"] = pd.to_datetime(f3_producto["fecha_reserva"], format='%d-%b-%Y')  # cambio de tipo de dato


In [3]:
f3_producto = f3_producto.loc[f3_producto.fecha_reserva > '01-01-2021'].reset_index()
f3_producto.fecha_reserva.dt.year.unique()

array([2021, 2022], dtype=int64)

In [4]:
f3_producto  = f3_producto.loc[(f3_producto ['descripcion6']=='enviado') | (f3_producto ['tipo_producto']=='producto')].reset_index()

In [5]:
f3_producto=f3_producto.sort_values("fecha_reserva")

In [6]:
f3_producto['Mes']  = f3_producto["fecha_reserva"].dt.strftime('%b-%y')

In [7]:
from datetime import date, timedelta
fecha_vencimiento = date.today() - timedelta(days=90)


In [8]:
f3_producto.loc[f3_producto["fecha_reserva"] < pd.to_datetime(fecha_vencimiento), 'Grupo_mes'] = 'Mayor a 90 días'

In [9]:
f3_producto.loc[f3_producto["fecha_reserva"] >= pd.to_datetime(fecha_vencimiento), 'Grupo_mes'] = f3_producto.loc[f3_producto["fecha_reserva"] >= pd.to_datetime(fecha_vencimiento), 'Mes'].values

In [10]:
grupo_90= f3_producto.groupby(["Mes",'Grupo_mes'], sort =False)['cant*costoprmd'].sum().reset_index()
grupo_90['color']=grupo_90['Grupo_mes']
grupo_90.loc[grupo_90['Grupo_mes']!= "Mayor a 90 días", "color"] = "Menos a 90 días"
grupo_90['cant*costoprmd']=round((grupo_90['cant*costoprmd']/1e6),2)

In [11]:
#f3_producto.to_excel(f'{dt_string}_f3_producto_enviado.xlsx')

In [12]:
grafica_prd=px.bar(grupo_90, x="Mes", y="cant*costoprmd", labels={'cant*costoprmd':'Costo'}, text='cant*costoprmd', color = 'color', color_discrete_sequence=["red", "blue"], title= "F3 Abiertos" )

grafica_prd.show()


In [13]:
grafica_prd.write_image("grafica_prd.jpg")

In [14]:
mayor_90= f3_producto.loc[f3_producto.Grupo_mes == "Mayor a 90 días"].groupby('Grupo_mes').agg({'cant*costoprmd':'sum', 'nro_devolucion':'nunique'})
mayor_90["cant*costoprmd"] = round((mayor_90["cant*costoprmd"]/1e6),2)
mayor_90 = mayor_90['cant*costoprmd'].sum()
mayor_90

12128.43

In [15]:
f3_producto["cant*costoprmd"] = ((f3_producto["cant*costoprmd"]/1e6))
total_prd= f3_producto.groupby('Mes').agg({'cant*costoprmd':'sum', 'nro_devolucion':'nunique'})
total_prd=f3_producto['cant*costoprmd'].sum()
costo_t = round(total_prd,2)
costo_t


15949.77

In [16]:
from smtplib import SMTP
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from email.mime.base import MIMEBase
from email.mime.application import MIMEApplication
#from python.llave import login_email as lg 

#def header_email():
mensaje = MIMEMultipart("related")
mensaje["From"]     = 'ext_angsanabria@falabella.com.co'
mensaje["To"]       = 'ext_angsanabria@falabella.com.co'  
mensaje["Subject"]  = 'MONITOREO F3 ENVIADO TIPO PRODUCTO '
#return mensaje

#def body():
email_content = f"""
                    <html>
                        <head>
                            <body>
                                <p> Estimados Oscar y Luis: </p>
                                <p> Por favor encuentren a continuación el análisis de F3 tipo producto en estado enviado para locales y cd, con su respectiva antigüedad.</p>
                                <p> -	Se evidencia un total de ${total_prd} M abiertos del 2021 de los cuales ${mayor_90} M tienen antigüedad mayor a 90 días</p>
                                <img src="cid:image1">
                                <p> El detalle de la información se encuentra adjunto.  </p>
                                <p> Quedo atento en caso de que surja alguna duda con la información anterior.</p>
                                <p> Saludos cordiales,</p>
                            </body>
                        </head>
                    """
    #return email_content

#def build_body(email_content,mensaje):
msgAlternative = MIMEMultipart('alternative') 
mensaje.attach(msgAlternative)
msgText = MIMEText(email_content, 'html') 
msgAlternative.attach(msgText)
fp = open('grafica_prd.jpg', 'rb') 
msgImage = MIMEImage(fp.read()) 
fp.close()

arch = open('220128_f3_producto_enviado.xlsx','rb')
msgAttach = MIMEApplication(arch.read())
arch.close()
msgAttach.add_header('Content-Disposition', '<excel1>')

#Adjuntando excel
part = MIMEBase('application', "octet-stream")
part.set_payload(open("220128_f3_producto_enviado.xlsx", "rb").read())#TODO CAMBIAR ARCHIVO
encode_base64(part)
part.add_header('Content-Disposition', 'attachment; filename="220128_f3_producto_enviado.xlsx"')
mensaje.attach(part)


msgImage.add_header('Content-ID', '<image1>') 
mensaje.attach(msgImage)
mensaje.attach(msgAttach)

#def send_email(mensaje):
smtp = SMTP("smtp.live.com", 587)
smtp.starttls()
smtp.login("ext_angsanabria@falabella.com.co","Paola1997")
smtp.sendmail("ext_angsanabria@falabella.com.co","ext_angsanabria@falabella.com.co",mensaje.as_string())
smtp.quit() 


(221, b'2.0.0 Service closing transmission channel')